# Gradient Boosted Tree Model with CatBoost

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, classification_report
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
from xgboost import to_graphviz
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import numpy as np
import graphviz
import joblib

In [38]:
# Cargar los datos preprocesados y las etiquetas
X = pd.read_csv("../../Data/preprocessed_training_simple_dataset.csv", index_col="respondent_id")
y = pd.read_csv("../../Data/training_set_labels.csv", index_col="respondent_id", usecols=["respondent_id", "h1n1_vaccine", "seasonal_vaccine"])

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [40]:
# Espacio de búsqueda para BayesSearchCV
search_space = {
    'estimator__depth': Integer(3, 10),  # Profundidad del árbol
    'estimator__learning_rate': Real(0.01, 0.3, prior='log-uniform'),  # Tasa de aprendizaje
    'estimator__l2_leaf_reg': Real(1, 10, prior='log-uniform'),  # Regularización L2
    'estimator__bagging_temperature': Real(0.0, 1.0),  # Temperatura de bagging
    'estimator__iterations': Integer(50, 500),  # Número de iteraciones
    'estimator__random_strength': Real(0.0, 10.0),  # Fuerza de aleatoriedad
    'estimator__auto_class_weights': Categorical(['Balanced', 'SqrtBalanced']),  # Pesos automáticos para clases
}

In [41]:
# Modelo base: CatBoostClassifier
catboost_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    verbose=0,
    random_seed=42
)

# MultiOutputClassifier para manejar múltiples etiquetas
multi_catboost = MultiOutputClassifier(catboost_model)

In [42]:
# Configurar BayesSearchCV
opt = BayesSearchCV(
    estimator=multi_catboost,
    search_spaces=search_space,
    cv=5,
    n_iter=50,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42
)

# Entrenar el modelo con BayesSearchCV
opt.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", opt.best_params_)
print("Mejor puntaje (AUROC):", opt.best_score_)

Mejores hiperparámetros: OrderedDict({'estimator__auto_class_weights': 'SqrtBalanced', 'estimator__bagging_temperature': 0.0, 'estimator__depth': 6, 'estimator__iterations': 500, 'estimator__l2_leaf_reg': 10.0, 'estimator__learning_rate': 0.024272428551470552, 'estimator__random_strength': 0.0})
Mejor puntaje (AUROC): 0.8653395384051465


In [43]:
# Predicciones de probabilidad en el conjunto de prueba
y_proba = opt.best_estimator_.predict_proba(X_test)

# Extraer probabilidades para cada etiqueta
y_proba_h1n1 = y_proba[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal = y_proba[1][:, 1]  # Probabilidades para vacuna estacional

# Calcular AUROC para cada etiqueta
roc_auc_h1n1 = roc_auc_score(y_test['h1n1_vaccine'], y_proba_h1n1)
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], y_proba_seasonal)

print(f"AUROC para H1N1: {roc_auc_h1n1}")
print(f"AUROC para vacuna estacional: {roc_auc_seasonal}")

# Predicciones finales
y_pred = opt.best_estimator_.predict(X_test)

# Informe de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

AUROC para H1N1: 0.8769076921304853
AUROC para vacuna estacional: 0.8619439180526232
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.64      0.65      0.65      1135
           1       0.77      0.76      0.77      2488

   micro avg       0.73      0.73      0.73      3623
   macro avg       0.71      0.70      0.71      3623
weighted avg       0.73      0.73      0.73      3623
 samples avg       0.35      0.36      0.35      3623



c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VEN

In [44]:
# Guardar el mejor modelo
joblib.dump(opt.best_estimator_, "Modelos/CatBoost_Best_Model_Bayes.pkl")
print("Modelo guardado como 'CatBoost_Best_Model_Bayes.pkl'")

Modelo guardado como 'CatBoost_Best_Model_Bayes.pkl'


In [45]:
# Cargar el dataset de prueba preprocesado
test_set_preprocessed = pd.read_csv("../../Data/preprocessed_test_simple_dataset.csv")

# Asegurarse de que respondent_id esté disponible
respondent_ids = test_set_preprocessed['respondent_id']

# Eliminar respondent_id del conjunto de características
X_test_final = test_set_preprocessed.drop(columns=['respondent_id'])

# Realizar predicciones de probabilidad
y_proba_test = opt.best_estimator_.predict_proba(X_test_final)

# Extraer las probabilidades para la clase positiva (1)
y_proba_h1n1_test = y_proba_test[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal_test = y_proba_test[1][:, 1]  # Probabilidades para vacuna estacional

# Crear el DataFrame de submission
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

# Guardar el archivo de submission
submission.to_csv("Submissions/CatBoost_Bayes_submission.csv", index=False)
print("Archivo 'CatBoost_Bayes_submission.csv' generado correctamente.")

Archivo 'CatBoost_Bayes_submission.csv' generado correctamente.
